## Fetch Songs from Spotify

### Authentication

### Initialize SpotiPy with user credentials saved in file

In [ ]:
# file = open("secrets.txt","r")
# content = file.read()
# secrets_dict={}

# for line in content.split('\n'):
#     if len(line) > 0:
#         secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()


# sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['ClientID'],
#                                                            client_secret=secrets_dict['ClientSecret']))

### Initialize SpotiPy with user credentials given in form

In [ ]:
ClientID = input('ClientID:').strip()
ClientSecret = input('ClientSecret:').strip()

#Initialize SpotiPy with user credentials using given input 
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=ClientID,
                                                           client_secret=ClientSecret))

### Playlists

In [ ]:
play_list_id = "1hMzceeWw7QiI6vaBkcEJO"

playlist = sp.user_playlist_tracks("spotify", play_list_id)

In [ ]:
playlist["total"] 

In [ ]:
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) # respectful nap
    return tracks

In [ ]:
all_tracks = get_playlist_tracks(play_list_id)

In [ ]:
tracks = json_normalize(all_tracks)

In [ ]:
# we can even create a new column containing these DataFrames! KEEP song URI
# NOTE: need to change this function, since the columns names are now a bit different
def expand_list_dict(row):
    df = json_normalize(row['track.artists'])
    df['song_id'] = row['track.id']
    return df

tracks['df_artists_by_song'] = tracks.apply(expand_list_dict, axis=1)

In [ ]:
# now we create a new dataframe with all these artists
df_artist = pd.DataFrame(columns=['href', 'id', 'name', 'type', 'uri'])
for mini_df in tracks['df_artists_by_song']:
    df_artist = pd.concat([df_artist, mini_df], axis=0)

In [ ]:
# now we merge (join) the two dataframes, keeping only the fields we need
df_merged = pd.merge(left=tracks,
                    right=df_artist,
                    how='inner',
                    left_on='track.id',
                    right_on='song_id')

In [ ]:
df_final = df_merged[['track.name', 'name', 'song_id']]